# Logisitic Regression, Linear SVC, Gaussian Naive Bayes

### Reading in the Data

In [20]:
import pipeline as pl
import imp

import pandas as pd
import numpy as np
import random
import time

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
imp.reload(pl)
claims = pl.read_all()

In [22]:
claims.head()

,Unnamed: 0,reimb2010,reimb2008,reimb2009,age2010,male,race,heart.failure,kidney,cancer,...,OfficeVisit,EyeExam,EKG,xray,CTScan,PhysicalTherapy,Ambulance,acuity,costTrend,monthsWithClaims
0,0,390,320.0,360,97,0,White,0,0,0,...,4,1,0,1,0,0,0,0.235294,-0.018856,9
1,1,970,58800.0,2740,79,0,White,1,1,0,...,12,0,2,8,1,0,1,0.853591,-0.027265,21
2,2,5630,510.0,1580,87,0,White,1,0,0,...,10,0,1,2,1,1,0,0.175115,0.496742,20
3,3,3480,2930.0,49330,79,0,White,1,1,0,...,14,1,1,6,4,3,3,0.709147,0.214955,19
4,4,920,1500.0,1650,85,1,White,1,0,1,...,16,2,1,3,0,1,0,0.174603,0.231568,21


### Exploring Data

In [23]:
pl.stats(claims)

Shape of the dataframe is (1023948, 29)


,Unnamed: 0,reimb2010,reimb2008,reimb2009,age2010,male,heart.failure,kidney,cancer,copd,...,OfficeVisit,EyeExam,EKG,xray,CTScan,PhysicalTherapy,Ambulance,acuity,costTrend,monthsWithClaims
count,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,...,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06,1.023948e+06
mean,5.119755e+05,2.843425e+03,4.648829e+03,4.919452e+03,7.318483e+01,4.325093e-01,5.044504e-01,3.157953e-01,1.277887e-01,2.612935e-01,...,9.863529e+00,1.130978e+00,1.410392e+00,2.454312e+00,9.224609e-01,3.446885e+00,1.223081e+00,3.751325e-01,8.071224e-02,1.449090e+01
std,2.955899e+05,5.959795e+03,1.043017e+04,8.954888e+03,1.237868e+01,4.954243e-01,4.999804e-01,4.648321e-01,3.338546e-01,4.393397e-01,...,8.518056e+00,1.650594e+00,1.859028e+00,3.023727e+00,1.449471e+00,5.442233e+00,3.178825e+00,2.586138e-01,2.073680e-01,8.070300e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.166667e-02,-7.627773e-01,0.000000e+00
25%,2.559878e+05,2.400000e+02,2.800000e+02,5.700000e+02,6.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.841004e-01,-4.075997e-02,9.000000e+00
50%,5.119755e+05,1.120000e+03,1.350000e+03,2.020000e+03,7.400000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,9.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,3.049738e-01,7.135935e-02,1.700000e+01
75%,7.679632e+05,2.620000e+03,3.800000e+03,4.970000e+03,8.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,...,1.500000e+01,2.000000e+00,2.000000e+00,4.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,5.272727e-01,2.390716e-01,2.100000e+01
max,1.023951e+06,1.605000e+05,2.307700e+05,1.870700e+05,1.000000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,6.100000e+01,2.300000e+01,2.800000e+01,4.100000e+01,2.100000e+01,1.330000e+02,1.260000e+02,1.000000e+00,8.179267e-01,2.400000e+01


In [24]:
claims.columns

Index(['Unnamed: 0', 'reimb2010', 'reimb2008', 'reimb2009', 'age2010', 'male',
       'race', 'heart.failure', 'kidney', 'cancer', 'copd', 'depression',
       'diabetes', 'ihd', 'osteoporosis', 'arthritis', 'stroke',
       'InpatientClaims', 'OutpatientClaims', 'OfficeVisit', 'EyeExam', 'EKG',
       'xray', 'CTScan', 'PhysicalTherapy', 'Ambulance', 'acuity', 'costTrend',
       'monthsWithClaims'],
      dtype='object')

### Pre-Processing Data

We will be trying OfficeVisit, costTrend, InpatientClaims, OutpatientClaims, age2010, reimb2009, reimb2008, heart.failure, and kidney as features to predict the outcome variable, which is reimb2010.

In [25]:
df = claims[['reimb2010', 'OfficeVisit', 'costTrend', 'InpatientClaims', 'OutpatientClaims', 'age2010', 
            'reimb2009', 'reimb2008', 'heart.failure', 'kidney']]

In [26]:
#fill missing columns
pl.fill_all_missing(df)

/Users/kellyyang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [31]:
train, test = pl.train_test(df, 0.2)

In [32]:
#normalize feature columns
train_df, test_df = pl.normalize(train, test, ['OfficeVisit', 'costTrend', 'InpatientClaims', 'OutpatientClaims', 'age2010', 
            'reimb2009', 'reimb2008', 'heart.failure', 'kidney'])

In [41]:
# Config: Dictionaries of models and hyperparameters - taken from class assignment
# MODELS = {
#     'LogisticRegression': LogisticRegression(), 
#     'LinearSVC': LinearSVC(), 
#     'GaussianNB': GaussianNB()
# }

# GRID = {
#     'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
#                            for x in ('l2', 'none') \
#                            for y in (0.01, 0.1, 1, 10, 100)],
#     'GaussianNB': [{'priors': None}],
#     'LinearSVC': [{'C': x, 'random_state': 0} \
#                   for x in (0.01, 0.1, 1, 10, 100)]
# }
MODELS = {
    'LogisticRegression': LogisticRegression(), 
    'LinearSVC': LinearSVC(), 
    'GaussianNB': GaussianNB()
}

GRID = {
    'LogisticRegression': [{'penalty': 'l2', 'C': 0.1, 'random_state': 0}],
    'GaussianNB': [{'priors': None}],
    'LinearSVC': [{'C': 1, 'random_state': 0}]
}

In [ ]:
results = pl.grid_search(train_df, test_df, MODELS, GRID, 'reimb2010') #kills the kernel running this

Training model: LogisticRegression | {'penalty': 'l2', 'C': 0.1, 'random_state': 0}
